In [3]:
!pip install -q google-genai

from google import genai
import json
import re
import kaggle_secrets

# Load Gemini API key from Kaggle Secrets
GEMINI_API_KEY = kaggle_secrets.UserSecretsClient().get_secret("GEMINI_API_KEY")

# Initialize client
client = genai.Client(api_key=GEMINI_API_KEY)

print("API Loaded Successfully")


API Loaded Successfully


In [4]:
def ask_gemini(prompt, model="gemini-2.5-flash-lite"):
    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"


In [5]:
def medical_ai(user_text):
    
    # STEP 1: Extract symptoms
    symptom_prompt = f"""
    Extract only medical symptoms from this text:
    "{user_text}"
    Return them as a Python list with lowercase names.
    Do not add extra text.
    """
    raw = ask_gemini(symptom_prompt)
    try:
        symptoms = eval(raw)
    except:
        symptoms = []
    
    # STEP 2: Disease Prediction
    disease_prompt = f"""
    Based on symptoms {symptoms}, predict possible illnesses.
    Give 1–3 possibilities max.
    Keep them short, like: ["viral fever", "flu"].
    """
    try:
        diseases = eval(ask_gemini(disease_prompt))
    except:
        diseases = ["Not enough data"]
    
    # STEP 3: Risk Level
    risk_prompt = f"""
    Symptoms: {symptoms}
    Determine a risk level: Low, Medium, or High.
    Rules:
    - breathlessness, chest pain → High
    - fever + cough + fatigue → Medium
    - mild symptoms → Low
    
    Output only one word: "Low", "Medium", or "High".
    """
    risk = ask_gemini(risk_prompt).replace("\n","")
    
    # STEP 4: Doctor Guidance
    doctor_prompt = f"""
    Based on symptoms {symptoms} and risk "{risk}", 
    write:
    - when they must see a doctor
    - emergency warning signs
    
    Keep it short.
    """
    doctor_advice = ask_gemini(doctor_prompt)
    
    # STEP 5: Home Remedies
    remedies_prompt = f"""
    Provide safe home remedies for symptoms {symptoms}.
    No dangerous or prescription drugs.
    """
    remedies = ask_gemini(remedies_prompt)
    
    # STEP 6: OTC Medicines (Safe Only)
    otc_prompt = f"""
    Based on symptoms {symptoms}, give safe OTC options.
    Rules:
    - paracetamol for fever/pain
    - ORS for dehydration
    - cough lozenges for sore throat
    - No antibiotics
    - No prescription drugs
    """
    otc = ask_gemini(otc_prompt)

    # FINAL STRUCTURED OUTPUT
    return {
        "symptoms_detected": symptoms,
        "possible_diseases": diseases,
        "risk_level": risk,
        "doctor_guidance": doctor_advice,
        "home_remedies": remedies,
        "safe_otc": otc
    }


In [6]:
user_input = "I have fever for two days, slight cough, and I feel very tired."
medical_ai(user_input)


{'symptoms_detected': ['fever', 'cough', 'tired'],
 'possible_diseases': ['common cold', 'flu'],
 'risk_level': 'Medium',
 'doctor_guidance': '**When to see a doctor:**\n\n*   If symptoms persist for more than a week or worsen.\n\n**Emergency warning signs:**\n\n*   Difficulty breathing\n*   Chest pain\n*   Confusion or extreme drowsiness\n*   Bluish lips or face',
 'home_remedies': "It's important to remember that these are home remedies and are not a substitute for professional medical advice. If your symptoms are severe, persistent, or you're concerned, please consult a doctor.\n\nHere are some safe home remedies for fever, cough, and tiredness:\n\n## For Fever:\n\n*   **Rest:** This is crucial for your body to fight off infection. Stay in bed and avoid strenuous activity.\n*   **Hydration:** Drink plenty of fluids to prevent dehydration and help your body regulate its temperature. Good choices include:\n    *   **Water:** The best choice.\n    *   **Herbal teas:** Chamomile, ginger

In [7]:
# Test Module for Medical AI
test_cases = [
    "I have fever, body pain and headache since morning.",
    "My throat is hurting and I have a dry cough.",
    "I feel weakness, dizziness and no appetite.",
    "Severe chest pain and shortness of breath.",
    "Runny nose and mild cough for two days."
]

for i, case in enumerate(test_cases, 1):
    print(f"\n--- TEST {i} ---")
    print("Input:", case)
    output = medical_ai(case)
    print("Output:", output)



--- TEST 1 ---
Input: I have fever, body pain and headache since morning.
Output: {'symptoms_detected': ['fever', 'body pain', 'headache'], 'possible_diseases': ['viral fever', 'flu', 'dengue'], 'risk_level': 'Low', 'doctor_guidance': "**See a Doctor If:**\n\n*   Symptoms worsen or don't improve after a few days.\n*   You have underlying health conditions.\n\n**Emergency Warning Signs:**\n\n*   Difficulty breathing\n*   Stiff neck\n*   Severe headache\n*   Rash", 'home_remedies': "It's important to remember that home remedies can help manage symptoms, but they don't replace professional medical advice. If your fever is high, persistent, or accompanied by other concerning symptoms, please consult a doctor.\n\nHere are some safe home remedies for fever, body pain, and headache:\n\n**For Fever:**\n\n*   **Rest:** This is crucial. Your body needs energy to fight off whatever is causing the fever. Stay in bed and avoid strenuous activities.\n*   **Hydration:** Drink plenty of fluids to pre

In [9]:
# --- TEST 5 ---
input_text = "I have vomiting and stomach cramps since last night."
print("--- TEST 5 ---")
print("Input:", input_text)
print("Output:", medical_ai(input_text))


--- TEST 5 ---
Input: I have vomiting and stomach cramps since last night.
Output: {'symptoms_detected': ['vomiting', 'stomach cramps'], 'possible_diseases': ['food poisoning', 'stomach flu'], 'risk_level': 'Low', 'doctor_guidance': '**See a doctor if:**\n\n*   Symptoms worsen or persist beyond 24 hours.\n*   You have a fever.\n\n**Emergency Warning Signs:**\n\n*   Severe abdominal pain.\n*   Blood in vomit or stool.\n*   Dehydration (dizziness, dry mouth, infrequent urination).\n*   Inability to keep fluids down.', 'home_remedies': "It's important to remember that while these home remedies can offer relief for mild symptoms, persistent or severe vomiting and stomach cramps can indicate a more serious underlying issue. If your symptoms are severe, don't improve within a day or two, or are accompanied by other concerning signs like fever, dehydration, blood in vomit or stool, or extreme pain, please **seek medical attention from a doctor or healthcare professional.**\n\nHere are some sa